# MCP Server Deployment on AgentCore Runtime

## 1. Environment Setup

### Setup Python Dependencies

In [ ]:
!printf "✅ Python version: $(python3 --version)\n"

from config import (
    SECRETS_NAME, PARAMETER_NAME,AGENT_NAME
)

!python3 -m pip install --requirement requirements.txt --upgrade --quiet

!echo "✅ Requirements installed"

## 2. Authentication Setup

### Setup Amazon Cognito User Pool

In [ ]:
from aws_setup import setup_cognito_user_pool

print("Setting up Amazon Cognito user pool (checking for existing pools first)...")
try:
    cognito_config = setup_cognito_user_pool()
    if cognito_config is None:
        raise Exception("Failed to setup Cognito user pool")
    print("✅ Cognito setup completed")
except Exception as e:
    print(f"❌ Error setting up Cognito: {e}")
    raise

### Display Amazon Cognito User Pool Configuration

In [ ]:
print(f'✅ Username: {cognito_config["AuthParameters"]["USERNAME"]}')
print(f'✅ PASSWORD: ***MASKED***')
print(f'✅ Bearer Token: ***MASKED***')
print(f'✅ Client ID: {cognito_config["client_id"]}')
print(f'✅ Pool ID: {cognito_config["pool_id"]}')
print(f'✅ Discovery URL: {cognito_config["discovery_url"]}')

## 3. AgentCore Runtime Deployment

### Configure AgentCore Runtime Hosting MCP Server

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session
import os

boto_session = Session()
region = boto_session.region_name
print(f"✅ Using AWS region: {region}")

required_files = ['awslabs/aws_support_mcp_server/server.py', 'requirements.txt']
try:
    for file in required_files:
        print(file)
        if not os.path.exists(file):
            raise FileNotFoundError(f"Required file {file} not found")
    print("✅ All required files found")
except FileNotFoundError as e:
    print(f"❌ {e}")
    raise

agentcore_runtime = Runtime()

if 'cognito_config' not in locals() or not cognito_config:
    raise Exception("Cognito configuration not available")

auth_config = {
    "customJWTAuthorizer": {
        "allowedClients": [
            cognito_config['client_id']
        ],
        "discoveryUrl": cognito_config['discovery_url'],
    }
}

print("Configuring AgentCore Runtime...")
response = agentcore_runtime.configure(
    entrypoint="awslabs/aws_support_mcp_server/server.py",
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    authorizer_configuration=auth_config,
    protocol="MCP",
    agent_name=AGENT_NAME
)
print("✅ Configuration completed")

### Launch AgentCore Runtime

In [ ]:
print("Launching MCP server to AgentCore Runtime (This might take several minutes...)")
launch_result = agentcore_runtime.launch(auto_update_on_conflict=True)
print("✅ Launch completed")

print(f"Agent ARN: {launch_result.agent_arn}")
print(f"Agent ID: {launch_result.agent_id}")

### Monitor AgentCore Runtime Status

In [ ]:
import time

print("Checking AgentCore Runtime status...")
status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
print(f"Initial status: {status}")

end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
max_wait_time = 1800  # 30 minutes
wait_time = 0

while status not in end_status and wait_time < max_wait_time:
    print(f"Status: {status} - waiting... ({wait_time}s/{max_wait_time}s)")
    time.sleep(10)
    wait_time += 10

    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']

if wait_time >= max_wait_time:
    print(f"⚠️ Timeout reached. Current status: {status}")
elif status == 'READY':
    print(f"✅ AgentCore Runtime is READY!")
else:
    print(f"⚠️ AgentCore Runtime status: {status}")
    
print(f"Final status: {status}")

## 4. Configuration Storage

### Store Configuration in AWS Secrets Manager and Parameter Store

In [ ]:
import boto3
import json

ssm_client = boto3.client('ssm', region_name=region)
secrets_client = boto3.client('secretsmanager', region_name=region)

try:
    cognito_credentials_response = secrets_client.create_secret(
        Name=SECRETS_NAME,
        Description='Cognito credentials for MCP server',
        SecretString=json.dumps(cognito_config)
    )
    print("✅ Cognito credentials stored in Secrets Manager")
except secrets_client.exceptions.ResourceExistsException:
    secrets_client.update_secret(
        SecretId=SECRETS_NAME,
        SecretString=json.dumps(cognito_config)
    )
    print("✅ Cognito credentials updated in Secrets Manager")
except Exception as e:
    print(f"❌ Error managing secrets: {e}")
    raise

try:
    agent_arn_response = ssm_client.put_parameter(
        Name=PARAMETER_NAME,
        Value=launch_result.agent_arn,
        Type='String',
        Description='Agent ARN for MCP server',
        Overwrite=True
    )
    print("✅ Agent ARN stored in Parameter Store")
except Exception as e:
    print(f"❌ Error storing Agent ARN: {e}")
    raise

print("\nConfiguration stored successfully!")
print(f"Agent ARN: {launch_result.agent_arn}")

## ✅ Deployment Complete

The MCP Server has been successfully deployed on AgentCore Runtime.

## 5. Testing & Validation

### Test MCP Server Connection and List Available Tools

In [ ]:
from mcp_client_test import test_mcp_connection
import asyncio

# Run the MCP client test
await test_mcp_connection()

## 6. Attach AWSSupportAccess IAM Policy to Agent Runtime Execution Role